<a href="https://colab.research.google.com/github/alejandratm18/TER/blob/main/TER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importation des différentes librairies utiles

In [1]:
import pandas as pd
import sys
!pip install python-Levenshtein
!pip install fuzzywuzzy
from Levenshtein import distance
from fuzzywuzzy import process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 14.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
my_local_drive='/content/gdrive/My Drive/Colab Notebooks/'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd

/content/gdrive/My Drive/Colab Notebooks


'/content/gdrive/My Drive/Colab Notebooks'

In [4]:
# Charger les fichiers CSV

#FakeNewsNet
df_fakenewsnetReal =pd.read_csv('Dataset/politifact_real.csv', sep=',')
display (df_fakenewsnetReal.head())

df_fakenewsnetFake =pd.read_csv('Dataset/politifact_fake.csv', sep=',')
display (df_fakenewsnetFake.head())

#ClaimsKG
df_claimsKGReal = pd.read_csv('Dataset/claimskg_Real.csv', sep=',')
display (df_claimsKGReal.head())

df_claimsKGFake = pd.read_csv('Dataset/claimskg_Fake.csv', sep=',')
display (df_claimsKGFake.head())

df_claimsKGMix = pd.read_csv('Dataset/claimskg_Mixture.csv', sep=',')
display (df_claimsKGMix.head())

df_claimsKGOther = pd.read_csv('Dataset/claimskg_Other.csv', sep=',')
display (df_claimsKGOther.head())


,id,news_url,title,tweet_ids
0,politifact14984,http://www.nfib-sbet.org/,National Federation of Independent Business,967132259869487105\t967164368768196609\t967215...
1,politifact12944,http://www.cq.com/doc/newsmakertranscripts-494...,comments in Fayetteville NC,942953459\t8980098198\t16253717352\t1668513250...
2,politifact333,https://web.archive.org/web/20080204072132/htt...,"Romney makes pitch, hoping to close deal : Ele...",NaN
3,politifact4358,https://web.archive.org/web/20110811143753/htt...,Democratic Leaders Say House Democrats Are Uni...,NaN
4,politifact779,https://web.archive.org/web/20070820164107/htt...,"Budget of the United States Government, FY 2008",89804710374154240\t91270460595109888\t96039619...


,id,news_url,title,tweet_ids
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...


,id,text,date,truthRating,ratingName,author,headline,named_entities_claim,named_entities_article,keywords,source,sourceURL,link,language
0,http://data.gesis.org/claimskg/claim_review/cc...,Individuals with mental illnesses die an avera...,2012-12-06,3,True,Terry Russell,NAMI Ohio leader Terry Russell says people wit...,"Affordable Care Act's,Gov. John Kasich,Nationa...","25 years earlier,mental illness,mental illnesses",http://data.gesis.org/claimskg/keyword/1cdbfb0...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2013/jan...,English
1,http://data.gesis.org/claimskg/claim_review/0c...,Texas is the state that has now gone the longe...,2012-09-22,3,True,Unknown,Joaquin Castro: Texas has gone the longest of ...,"Alabama,Arizona,Associated Press,Attorney Gene...","Democrat,Texas",http://data.gesis.org/claimskg/keyword/cc43933...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2012/sep...,English
2,http://data.gesis.org/claimskg/claim_review/5e...,New England already has access to ample suppli...,2010-11-04,3,True,Unknown,Senator Reed says New England has ample suppli...,"EIA,Energy Information Administration,Fall Riv...","New England,natural gas",http://data.gesis.org/claimskg/keyword/2fd138c...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2011/jan...,English
3,http://data.gesis.org/claimskg/claim_review/82...,I don't do earmarks. I've never done one. I'm ...,2009-03-12,3,True,Unknown,"Yes, Rep. John Boehner has never asked for ear...","Boehner,Conference Chairman,Eric Cantor,House ...",NaN,http://data.gesis.org/claimskg/keyword/d3d3502...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2009/mar...,English
4,http://data.gesis.org/claimskg/claim_review/83...,Wayne Powell has a stated position of having n...,2012-08-29,3,True,Unknown,Eric Cantor says Wayne Powell doesn't object t...,"Chesterfield County,Democrat,Democratic,Eric C...",U.S. currency,http://data.gesis.org/claimskg/keyword/91138a7...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2012/sep...,English


,id,text,date,truthRating,ratingName,author,headline,named_entities_claim,named_entities_article,keywords,source,sourceURL,link,language
0,http://data.gesis.org/claimskg/claim_review/4a...,"If you take the vaccine, you'll be enrolled in...",2021-02-06,1,False,Unknown,Receiving COVID-19 vaccine does not enroll you...,"Anonymized data,CDC,COVID-19 vaccine,COVID-19 ...","medical trial,pharmacovigilance,vaccine",http://data.gesis.org/claimskg/keyword/031df93...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2021/feb...,English
1,http://data.gesis.org/claimskg/claim_review/2f...,FEMA forcibly vaccinating Ida evacuees.,2021-09-01,1,False,Unknown,"No, FEMA isn’t forcing Ida evacuees to get a C...","Biden administration,CDC,COVID-19 test,COVID-1...",FEMA,http://data.gesis.org/claimskg/keyword/8620acf...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2021/sep...,English
2,http://data.gesis.org/claimskg/claim_review/15...,The stories from women saying he groped or for...,2016-10-19,1,False,Unknown,Donald Trump says reports of sexual assault by...,"Anthony Senecal,Calif.,California,Cassandra Se...",NaN,http://data.gesis.org/claimskg/keyword/3ac760f...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2016/oct...,English
3,http://data.gesis.org/claimskg/claim_review/a5...,California literally just told everyone to not...,2021-06-18,1,False,Unknown,Can Californians charge electric cars during s...,"Air conditioning units,CAISO,California,Califo...",California,http://data.gesis.org/claimskg/keyword/2fd138c...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2021/jul...,English
4,http://data.gesis.org/claimskg/claim_review/c5...,White men account for 69 percent of those arre...,2015-03-19,1,False,Unknown,Sally Kohn: 'White men account for 69 percent ...,"2012,4.7,African-American,Aggravated Assault,B...","White,violent crimes",http://data.gesis.org/claimskg/keyword/3ac760f...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2015/apr...,English


,id,text,date,truthRating,ratingName,author,headline,named_entities_claim,named_entities_article,keywords,source,sourceURL,link,language
0,http://data.gesis.org/claimskg/claim_review/01...,If you don't buy cigarettes at your local supe...,2016-04-14,2,MIXTURE,Philadelphia Daily News,Rating the Philly Daily News’ claim that ‘It’s...,"Carl Davis,Daily News,Diet Coke,Donald Marron,...","sugary drink tax,sugary drinks,supermarket",http://data.gesis.org/claimskg/keyword/62602a6...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2016/may...,English
1,http://data.gesis.org/claimskg/claim_review/de...,Says Ronald Reagan got the Saudis to flood the...,2014-03-06,2,MIXTURE,Michael Reagan,Ronald Reagan's son says his father got the Sa...,"1980s,1981,1985,1986,1987,1988,1992,Alternate ...",Ronald Reagan,http://data.gesis.org/claimskg/keyword/31f18fd...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2014/mar...,English
2,http://data.gesis.org/claimskg/claim_review/98...,One of the reasons I believe we’re in trouble ...,2015-06-14,2,MIXTURE,Unknown,Chafee says drone strikes drive the unrest in ...,"2009,Brookings Institution,Bruce Riedel,Chafee...",Yemen,http://data.gesis.org/claimskg/keyword/d2c37b9...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2015/jul...,English
3,http://data.gesis.org/claimskg/claim_review/7d...,The IRS will have the ability potentially to d...,2013-05-15,2,MIXTURE,Unknown,Michele Bachmann says IRS will have the abilit...,"2016,6 million,Affordable Care Act,Bachmann,Br...",IRS,http://data.gesis.org/claimskg/keyword/74e6b36...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2013/may...,English
4,http://data.gesis.org/claimskg/claim_review/1c...,Says Sen. Thad Cochran voted to spend hundreds...,2014-03-19,2,MIXTURE,Chris Chocola,Did Mississippi Sen. Thad Cochran spurn Katrin...,"1940 Ford,1972,2008 presidential race,5.5,Alas...","Alaska,Hurricane Katrina,Sen.,Thad Cochran,mil...",http://data.gesis.org/claimskg/keyword/2a15a9e...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2014/mar...,English


,id,text,date,truthRating,ratingName,author,headline,named_entities_claim,named_entities_article,keywords,source,sourceURL,link,language
0,http://data.gesis.org/claimskg/claim_review/0f...,On running a civil and polite campaign.,2010-08-11,-1,OTHER,Unknown,Georgia gubernatorial hopeful Roy Barnes said ...,"Action News,Brian Robinson,Democrat,Democratic...",NaN,http://data.gesis.org/claimskg/keyword/5db25fe...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2010/aug...,English
1,http://data.gesis.org/claimskg/claim_review/d1...,On the use of presidentially appointed commiss...,2008-09-17,-1,OTHER,Unknown,Obama flip-flops on use of presidential commis...,"Alan Simpson,Amy Gutmann,BP Deepwater Horizon ...",NaN,http://data.gesis.org/claimskg/keyword/2d6fbbd...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2010/jul...,English
2,http://data.gesis.org/claimskg/claim_review/cc...,On supporting Mitch McConnell as U.S. Senate m...,2018-07-16,-1,OTHER,Unknown,Does GOP U.S. Senate candidate Kevin Nicholson...,"Associated Press,Bannon,D.C.,Democrat,Facebook...","Mitch McConnell,U.S. Senate majority leader",http://data.gesis.org/claimskg/keyword/4edc7af...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2018/jul...,English
3,http://data.gesis.org/claimskg/claim_review/49...,On the Cuba embargo.,2015-07-31,-1,OTHER,Unknown,Did Hillary Clinton change position on the Cub...,"1996,2000 campaign,2008,2014,Brothers to the R...",Cuba,http://data.gesis.org/claimskg/keyword/71ade47...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2015/aug...,English
4,http://data.gesis.org/claimskg/claim_review/f4...,On transportation financing,2012-05-23,-1,OTHER,Unknown,Christie backtracks on financing transportatio...,"Alan M. Voorhees,Andrew Sidamon-Eristoff,Chris...",NaN,http://data.gesis.org/claimskg/keyword/d378579...,politifact,http://www.politifact.com,https://www.politifact.com/factchecks/2012/may...,English


In [45]:
#Creation de sous-ensemble des données. Mot clé: Barack Obama

resultClaimsReal = df_claimsKGReal[df_claimsKGReal['headline'].str.contains('Obama', case=False)]
resultClaimsFake = df_claimsKGFake[df_claimsKGFake['headline'].str.contains('Obama', case=False)]
resultClaimsMix = df_claimsKGMix[df_claimsKGMix['headline'].str.contains('Obama', case=False)]
resultClaimsOther = df_claimsKGOther[df_claimsKGOther['headline'].str.contains('Obama', case=False)]

#Pour montrer uniquement certains colonnes
resultClaimsReal= resultClaimsReal[['text', 'date', 'author', 'headline']]
resultClaimsFake= resultClaimsFake[['text', 'date', 'author', 'headline']]

#Pareil pour FakeNewsNet, creation de sous-ensemble des données. Mot clé: Barack Obama
resultFakenewsnetReal = df_fakenewsnetReal[df_fakenewsnetReal['title'].str.contains('Obama', case=False)]
resultFakenewsnetFake = df_fakenewsnetFake[df_fakenewsnetReal['title'].str.contains('Obama', case=False)]

#Dans ce cas, nous gardons uniquement la colonne "title"
resultFakenewsnetReal = resultFakenewsnetReal[['title']]
resultFakenewsnetFake = resultFakenewsnetFake[['title']]


<ipython-input-45-9a6e708b377d>:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  resultFakenewsnetFake = df_fakenewsnetFake[df_fakenewsnetReal['title'].str.contains('Obama', case=False)]


In [46]:
# Concaténer les textes des articles dans un seul DataFrame
ClaimsKG = pd.concat([resultClaimsReal['headline'], resultClaimsFake['headline'], resultClaimsMix['headline'], resultClaimsOther['headline']], ignore_index=True)

FakeNewsNet = pd.concat([resultFakenewsnetReal['title'], resultFakenewsnetFake['title']], ignore_index=True)

display (ClaimsKG.head())

display (FakeNewsNet.head())

0    Jerry Patterson says Barack Obama broke his pr...
1    Obama right that Roosevelt was called a social...
2    Bob Schieffer says Iran President Hassan Rouha...
3    Barack Obama says taxes are lower for middle c...
4    Barack Obama in State of the Union says corpor...
Name: headline, dtype: object

0                 Obamas comments on capandtrade costs
1                   Barack Obama’s Remarks in St. Paul
2    Ralph Nader on Barack Obama: "It is Quite Clea...
3    Strong Words in Ohio as Obama and Clinton Pres...
4    “Dictionary” on President Obama’s Health Care ...
Name: title, dtype: object

In [47]:
print(len(ClaimsKG))
print(len(FakeNewsNet))

2714
105


In [61]:
!pip install python-Levenshtein
from Levenshtein import distance

# Seuil de similarité
changes = 0

# Comparaison des articles
articles_pairs = set()
for headline in ClaimsKG:
    for title in FakeNewsNet:
        dist = distance(headline.lower(), title.lower())
        if dist == changes:
            articles_pairs.add((headline, title))

# Affichage des paires d'articles similaires
for pair in articles_pairs:
    print("ClaimsKG Article:", pair[0])
    print("FakeNewsNet Article:", pair[1])
    print()

ClaimsKG Article: Republican consultant says Barack Obama promised to halve federal deficit in his first term
FakeNewsNet Article: Republican consultant says Barack Obama promised to halve federal deficit in his first term

ClaimsKG Article: Cain said food stamp program use up under Obama
FakeNewsNet Article: Cain said food stamp program use up under Obama

ClaimsKG Article: Ad says Obama apologized, showed weakness on Iran
FakeNewsNet Article: Ad says Obama apologized, showed weakness on Iran

ClaimsKG Article: Barack Obama says Mitt Romney condemned coal-fired power plants as killers
FakeNewsNet Article: Barack Obama says Mitt Romney condemned coal-fired power plants as killers

ClaimsKG Article: Has the potency of pot changed since President Obama was in high school?
FakeNewsNet Article: Has the potency of pot changed since President Obama was in high school?

ClaimsKG Article: Barack Obama says U.S. oil production last year was highest since 2003
FakeNewsNet Article: Barack Obama s

**Pipeline**


---



In [67]:
import pandas as pd
from Levenshtein import distance

 # Fonction pour filtrer les données par entité/topic

def filtrer(dataframe, topic):
    return dataframe[dataframe.str.contains(topic, case=False)]


 # Fonction pour comparer les articles/ trouver similitudes

def comparer_articles(claimsKG_data, fakeNewsNet_data, similarity_threshold):
    similar_pairs = set()
    for headline in claimsKG_data:
        for title in fakeNewsNet_data:
            dist = distance(headline.lower(), title.lower())
            if dist <= similarity_threshold:
                similar_pairs.add((headline, title))
    return similar_pairs

 # Fonction pour montrer les pairs d'articles similaires
def articles_similairs(similar_pairs):
    for pair in similar_pairs:
        print("ClaimsKG Article:", pair[0])
        print("FakeNewsNet Article:", pair[1])
        print()

 # Fonction qui execute la pipeline
def lancer_pipeline(topic, similarity_threshold):
    # Filtrer par thème
    claimsKG_filtered = filtrer(ClaimsKG, topic)
    fakeNewsNet_filtered = filtrer(FakeNewsNet, topic)

    # Trouver pairs
    similar_pairs = comparer_articles(claimsKG_filtered, fakeNewsNet_filtered, similarity_threshold)

    # Montrer resultats
    articles_similairs(similar_pairs)

# Configuration
topic = "Obama"
similarity_threshold = 0

# Executer la pipeline
lancer_pipeline(topic, similarity_threshold)


ClaimsKG Article: Republican consultant says Barack Obama promised to halve federal deficit in his first term
FakeNewsNet Article: Republican consultant says Barack Obama promised to halve federal deficit in his first term

ClaimsKG Article: Cain said food stamp program use up under Obama
FakeNewsNet Article: Cain said food stamp program use up under Obama

ClaimsKG Article: Ad says Obama apologized, showed weakness on Iran
FakeNewsNet Article: Ad says Obama apologized, showed weakness on Iran

ClaimsKG Article: Barack Obama says Mitt Romney condemned coal-fired power plants as killers
FakeNewsNet Article: Barack Obama says Mitt Romney condemned coal-fired power plants as killers

ClaimsKG Article: Has the potency of pot changed since President Obama was in high school?
FakeNewsNet Article: Has the potency of pot changed since President Obama was in high school?

ClaimsKG Article: Barack Obama says U.S. oil production last year was highest since 2003
FakeNewsNet Article: Barack Obama s